In [ ]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import nltk
nltk.download("all")
import nltk
nltk.download('punkt')

import matplotlib.pyplot as plt
import torch

%matplotlib inline

In [4]:
from google.colab import drive

#get datasets
drive.mount('/content/drive', force_remount=True)

# change the foldername as the project folder location
FOLDERNAME ='/content/drive/MyDrive/CS329P/Project'

# assert FOLDERNAME is not None, "[!] Enter the foldername."
# %cd /content/drive/MyDrive
%cd $FOLDERNAME

Mounted at /content/drive
/content/drive/MyDrive/CS329P/Project


In [6]:
def load_text(path, type):

    """
    Load text data, convert all into lowercase text and save to a list.
    """
    if type == 1:

        with open(path, 'rb') as f:
            texts = []
            for line in f:
                texts.append(line.decode(errors='ignore').lower().strip())
    if type == 2:
        with open(path, 'rb') as f:
            texts = []
            for line in f:
                texts.append(int(line.decode(errors='ignore').lower().strip()))  
    return texts

# Load files
data_text = load_text('data_cnn/asset_indexdata.txt', 1)
labels = load_text('data_cnn/asset_indexlabel.txt',2)
# convert into np array
texts = np.array(data_text)
labels = np.array(labels)

In [7]:
print(texts, labels)

['as such during the late ottoman era, upper reka males (mainly adults) would seasonally go on kurbet or economic migration.. often they would find employment as pastry makers or as halva, salep and boza merchants and salesmen in the then ottoman capital istanbul or regional cities like skopje and edirne.'
 'in october 2012, privatization agency of serbia announced that it had won the case against the former owners, meaning they will have to pay €17 million in the name of compensation. however, in the early 2019 city announced new urban regulatory plan for the area where the brewery is, envisioning the conversion of the land from economic to commercial. offered assets include factory complex at mostar (), juice factory, brewery in čačak, commercial offices in obrenovac, kosovska mitrovica and alibunar, retail store in kragujevac, and an apartment in budva, montenegro. bogdan veljković described the entire sale process as criminal, lawless, corruption, claiming the entire worth of the b

In [13]:
print(type(labels[0]), type(labels1[0]))
print(type(texts[0]), type(texts1[0]))
print(type(labels), type(labels1))
print(type(texts), type(texts1))

<class 'numpy.int64'> <class 'numpy.int64'>
<class 'numpy.str_'> <class 'numpy.str_'>
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


Let's use pretrained word vectors(FastText Word Vectors, Mikolov et al., 2017) to train the tokens

In [14]:
%%time
URL = "https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip"
FILE = "fastText"

if os.path.isdir(FILE):
    print("fastText exists.")
else:
    !wget -P $FILE $URL
    !unzip $FILE/crawl-300d-2M.vec.zip -d $FILE



fastText exists.
CPU times: user 0 ns, sys: 441 µs, total: 441 µs
Wall time: 482 µs


In [15]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla P100-PCIE-16GB


# Tokenizing

Not sure whether we need this process or not - Cara already made some codes for this

In [16]:

from tqdm import tqdm_notebook
from collections import defaultdict
from nltk.tokenize import word_tokenize
from torch.utils.data import (TensorDataset, DataLoader, RandomSampler, SequentialSampler)
from sklearn.model_selection import train_test_split


def tokenize(texts):
    """
    1. Tokenize
    2. build vocabulary
    3. save maximum sentence length 
    Output:
        tokenized 
        word2idx (Dict): Vocabulary built from the corpus
        max_len 
    """
    max_len = 0
    tokenized_texts = []
    word2idx = {}

    # Add <pad> and <unk> tokens to the vocabulary
    word2idx['<pad>'] = 0
    word2idx['<unk>'] = 1

    # Building our vocab from the corpus starting from index 2
    idx = 2
    for sent in texts:
        tokenized_sent = word_tokenize(sent)

        # Add `tokenized_sent` to `tokenized_texts`
        tokenized_texts.append(tokenized_sent)

        # Add new token to `word2idx`
        for token in tokenized_sent:
            if token not in word2idx:
                word2idx[token] = idx
                idx += 1

        # Update `max_len`
        max_len = max(max_len, len(tokenized_sent))

    return tokenized_texts, word2idx, max_len

def encode(tokenized_texts, word2idx, max_len):
    """
    First, we need to pad each sentence to the number of maximum sentence length
    Next, encode tokens to the index 

    Output: Input for the model - array of token indexes in the vocabulary, (N, max_len)
    """

    input_ids = []
    for tokenized_sent in tokenized_texts:
        # Pad sentences to max_len
        tokenized_sent += ['<pad>'] * (max_len - len(tokenized_sent))

        # Encode tokens to input_ids
        input_id = [word2idx.get(token) for token in tokenized_sent]
        input_ids.append(input_id)
    
    return np.array(input_ids)

def load_pretrained_vectors(word2idx, fname):
    """
    Input:
        word2idx (Dict): Vocabulary built from the corpus
        fname: Path to pretrained vector file

    Output:
        embeddings (np.array): Embedding matrix with shape (N, d) where N is
            the size of word2idx and d is embedding dimension
    """

    print("Loading pretrained vectors...")
    fin = open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())


    embeddings = np.random.uniform(-0.25, 0.25, (len(word2idx), d))
    embeddings[word2idx['<pad>']] = np.zeros((d,))

    # Load pretrained vectors
    count = 0
    for line in tqdm_notebook(fin):
        tokens = line.rstrip().split(' ')
        word = tokens[0]
        if word in word2idx:
            count += 1
            embeddings[word2idx[word]] = np.array(tokens[1:], dtype=np.float32)

    print(f"There are {count} / {len(word2idx)} pretrained vectors found.")

    return embeddings

In [39]:
import nltk
nltk.download('punkt')

################ Tokenize
tokenized_texts, word2idx, max_len = tokenize(texts)
input_ids = encode(tokenized_texts, word2idx, max_len)

# Pretrained vectors
embeddings = load_pretrained_vectors(word2idx, "fastText/crawl-300d-2M.vec")
embeddings = torch.tensor(embeddings)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Loading pretrained vectors...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:89: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

There are 6335 / 8130 pretrained vectors found.


In [40]:

def data_loader(train_inputs, val_inputs, train_labels, val_labels,
                batch_size=50):

    train_inputs, val_inputs, train_labels, val_labels =\
    tuple(torch.tensor(data) for data in
          [train_inputs, val_inputs, train_labels, val_labels])

    ############ batch
    batch_size = 50

    # DataLoader for training data
    train_data = TensorDataset(train_inputs, train_labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    # DataLoader for validation data
    val_data = TensorDataset(val_inputs, val_labels)
    val_sampler = SequentialSampler(val_data)
    val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

    return train_dataloader, val_dataloader

In [41]:
# Train Test Split
train_inputs, val_inputs, train_labels, val_labels = train_test_split(
    input_ids, labels, test_size=0.1, random_state=42)

# Load data to PyTorch DataLoader
train_dataloader, val_dataloader = data_loader(train_inputs, val_inputs, train_labels, val_labels, batch_size=50)

In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import time

# Sample configuration:
filter_sizes = [2, 3, 4]
num_filters = [2, 2, 2]


# 1D CNN
class CNN_NLP(nn.Module):
    def __init__(self,
                 pretrained_embedding=None,
                 freeze_embedding=False,
                 vocab_size=None,
                 embed_dim=300,
                 filter_sizes=[3, 4, 5],
                 num_filters=[100, 100, 100],
                 num_classes=2,
                 dropout=0.5):
        """
            pretrained_embedding: (vocab_size, embed_dim)

            When pretrained word embeddings are not used
                vocab_size,  embed_dim
            n_classes: for now, we simplified the class as two (idx = 1 or idx = 2, because higher index has small # of dataset)
            dropout rate
        """

        super(CNN_NLP, self).__init__()
        
        
        # Embedding layer when we use pretrained model 
        if pretrained_embedding is not None:
            self.vocab_size, self.embed_dim = pretrained_embedding.shape
            self.embedding = nn.Embedding.from_pretrained(pretrained_embedding,
                                                          freeze=freeze_embedding)
        else:
            self.embed_dim = embed_dim
            self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                          embedding_dim=self.embed_dim,
                                          padding_idx=0,
                                          max_norm=5.0)
        # Conv Network
        self.conv1d_list = nn.ModuleList([
            nn.Conv1d(in_channels=self.embed_dim,
                      out_channels=num_filters[i],
                      kernel_size=filter_sizes[i])
            for i in range(len(filter_sizes))
        ])
        # Fully-connected layer and Dropout
        self.fc = nn.Linear(np.sum(num_filters), num_classes)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, input_ids):


        #  (b, max_len, embed_dim)
        embed_frominput = self.embedding(input_ids).float()

        #  (b, embed_dim, max_len)
        embed_reshaped = embed_frominput.permute(0, 2, 1)

        # CNN & ReLU: (b, num_filters[i], L_out)
        embed_conv_list = [F.relu(conv1d(embed_reshaped)) for conv1d in self.conv1d_list]

        # Max pool: (b, num_filters[i], 1)
        embed_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2])
            for x_conv in embed_conv_list]
        
        # FC- (b, sum(num_filters))
        embed_fc = torch.cat([x_pool.squeeze(dim=2) for x_pool in embed_pool_list],
                         dim=1)
        
        # (b, n_classes)
        output = self.fc(self.dropout(embed_fc))

        return output

In [43]:
def initialize_model(pretrained_embedding=None, freeze_embedding=False,
                    vocab_size=None,  embed_dim=300,
                    filter_sizes=[3, 4, 5],
                    num_filters=[100, 100, 100],
                    num_classes=2,
                    dropout=0.5,
                    learning_rate=0.01):

    assert (len(filter_sizes) == len(num_filters)), "filter_sizes and num_filters are not the same length!"

    cnn_model = CNN_NLP(pretrained_embedding=pretrained_embedding,
                        freeze_embedding=freeze_embedding,
                        vocab_size=vocab_size,
                        embed_dim=embed_dim,
                        filter_sizes=filter_sizes,
                        num_filters=num_filters,
                        num_classes=2,
                        dropout=0.5)
    
    cnn_model.to(device)

    # since it is adam, maybe we need to decrease lr into 0.0001?
    optimizer = optim.Adam(cnn_model.parameters(),
                               lr=learning_rate)

    return cnn_model, optimizer


loss_fn = nn.CrossEntropyLoss()

# for reproducability, set random seed
def set_seed(seed_value=32):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)


def train(model, optimizer, train_dataloader, val_dataloader=None, epochs=10):
    """Train the CNN model."""
    
    # Tracking best validation accuracy
    best_accuracy = 0

    print("Start training!\n")

    print(f"{'Epoch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {\
    'Val Acc':^9} | {'Elapsed':^9}")
    print("-"*60)

    for epoch_i in range(epochs):


        ################## Training ##############

        t0_epoch = time.time()
        total_loss = 0

        # training mode
        model.train()

        for step, batch in enumerate(train_dataloader):
            # Load batch to GPU
            tr_input_ids, tr_labels = tuple(t.to(device) for t in batch)

            model.zero_grad()
            pred_grad = model(tr_input_ids)

            loss = loss_fn(pred_grad, tr_labels)
            total_loss += loss.item()

            loss.backward()

            optimizer.step()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        ################## Evaluation #######################

        if val_dataloader is not None:

            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # best accuracy
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy

            time_elapsed = time.time() - t0_epoch
            print(f"{epoch_i + 1:^7} | {avg_train_loss:^12.6f} | {\
            val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            
    print("\n")
    print(f"Training complete! Best accuracy: {best_accuracy:.2f}%.")

def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's
    performance on our validation set.
    """

    model.eval()

    val_accuracy = []
    val_loss = []

    for batch in val_dataloader:
        ev_input_ids, ev_labels = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            logits = model(ev_input_ids)

        loss = loss_fn(logits, ev_labels)
        val_loss.append(loss.item())

        preds = torch.argmax(logits, dim=1).flatten()

        # Accuracy rate from mean
        accuracy = (preds == ev_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

In [44]:
# are randomly initialized word vectors
cnn_rand, optimizer = initialize_model(vocab_size=len(word2idx),
                                      embed_dim=300,
                                      learning_rate=0.25,
                                      dropout=0.5)
train(cnn_rand, optimizer, train_dataloader, val_dataloader, epochs=20)

Start training!

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------
   1    |  71.563192   | 125.822837 |   89.23   |   1.56   
   2    |  300.847950  | 1054.713114 |   63.85   |   1.54   
   3    |  322.448506  | 182.718499 |   95.84   |   1.54   
   4    |  271.451195  | 417.557353 |   96.22   |   1.54   
   5    |  225.392301  | 259.137407 |   96.22   |   1.53   
   6    |  290.069156  | 324.013992 |   97.46   |   1.55   
   7    |  231.990994  | 587.068177 |   96.89   |   1.53   
   8    |  277.651063  | 167.044724 |   98.89   |   1.53   
   9    |  378.204497  | 1504.912021 |   96.73   |   1.54   
  10    |  306.045608  | 806.691069 |   98.00   |   1.54   
  11    |  465.283389  | 600.442376 |   98.44   |   1.54   
  12    |  350.056999  | 485.382697 |   99.11   |   1.53   
  13    |  226.516322  | 633.442813 |   99.33   |   1.54   
  14    |  317.766670  | 998.012990 |   98.67   |   1.54   
  15    |  143.83517

In [45]:
# freezed pretrained word vectors during training
set_seed(28)
cnn_static, optimizer = initialize_model(pretrained_embedding=embeddings,
                                        freeze_embedding=True,
                                        learning_rate=0.25,
                                        dropout=0.5)
train(cnn_static, optimizer, train_dataloader, val_dataloader, epochs=20)

Start training!

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------
   1    |  20.039886   |  7.277438  |   87.39   |   1.06   
   2    |  14.194124   |  7.735140  |   90.41   |   1.06   
   3    |  22.819833   | 20.685639  |   90.91   |   1.06   
   4    |  33.215984   | 17.257021  |   96.06   |   1.06   
   5    |  37.800380   | 27.668060  |   96.51   |   1.06   
   6    |  37.595427   | 10.824505  |   98.89   |   1.06   
   7    |  19.223685   | 17.624871  |   97.39   |   1.06   
   8    |  18.740821   |  9.220479  |   98.89   |   1.06   
   9    |  27.766165   | 10.738818  |   99.11   |   1.06   
  10    |  38.282627   | 10.148971  |   98.73   |   1.06   
  11    |  33.584561   | 17.236425  |   99.11   |   1.07   
  12    |  32.105178   | 26.532780  |   98.89   |   1.07   
  13    |  27.313973   |  9.601482  |   99.56   |   1.06   
  14    |  13.393506   |  9.772540  |   99.33   |   1.07   
  15    |  21.133523  

In [46]:
# fine-tuned pretrained word vectors during training 
set_seed(42)
cnn_non_static, optimizer = initialize_model(pretrained_embedding=embeddings,
                                            freeze_embedding=False,
                                            learning_rate=0.25,
                                            dropout=0.5)
train(cnn_non_static, optimizer, train_dataloader, val_dataloader, epochs=20)


Start training!

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------
   1    |  414.144132  | 2906.369646 |   91.17   |   1.59   
   2    | 3174.279405  | 3614.241252 |   97.11   |   1.59   
   3    | 1929.099525  | 3375.348715 |   99.33   |   1.59   
   4    | 1668.012666  | 5754.295736 |   98.44   |   1.59   
   5    | 4420.628805  | 26460.582275 |   97.78   |   1.59   
   6    | 9958.678696  | 8580.697266 |   99.56   |   1.59   
   7    | 6907.567107  | 33186.208767 |   98.73   |   1.59   
   8    | 2000.061381  | 21962.451565 |   99.11   |   1.59   
   9    |  779.760617  | 25240.001628 |   98.73   |   1.59   
  10    | 7229.381028  | 32643.407254 |   98.51   |   1.59   
  11    | 7303.048252  | 43878.338135 |   98.67   |   1.59   
  12    | 6015.980996  | 47747.253472 |   99.33   |   1.59   
  13    | 3031.479755  | 34899.798177 |   99.33   |   1.58   
  14    | 4641.112536  | 80103.300347 |   99.33   |   1.59   

In [47]:
def predict(text, model=cnn_non_static.to("cpu"), max_len=62):
    """Predict probability that a review is positive."""

    # Tokenize, pad and encode text
    tokens = word_tokenize(text.lower())
    padded_tokens = tokens + ['<pad>'] * (max_len - len(tokens))
    input_id = [word2idx.get(token, word2idx['<unk>']) for token in padded_tokens]

    # Convert to PyTorch tensors
    input_id = torch.tensor(input_id).unsqueeze(dim=0)

    # Compute logits
    logits = model.forward(input_id)

    #  Compute probability
    probs = F.softmax(logits, dim=1).squeeze(dim=0)

    print(f"This review is {probs[1] * 100:.2f}% positive.")

In [48]:
predict("The lower dam will regulate outflows from the 횉etin main dam and also produce hydroelectric power with a 112 MW capacity via two 56 MW Kaplan turbines.  .")

This review is 0.00% positive.
